# RK(3,2)-FB FBFC Analysis (Eigenvalue Optimization)

*FBFC = Forward-backward feedback coefficients*

Consider a simple model problem:
\begin{align*}
    \frac{\partial \zeta}{\partial t} &= -c \frac{\partial u}{\partial x}, \\
    \frac{\partial u}{\partial t} &= -c \frac{\partial \zeta}{\partial x}.
\end{align*}
Note that this is similar to the 1D linearized SWEs with the Coriolis term deleted.

With the goal of applying a von Neumann stability anaysis, we can apply a Fourier transform in $x$ to the above to get
\begin{align*}
    \frac{\partial \hat{\zeta}}{\partial t} &= -i \omega \hat{u}, \\
    \frac{\partial \hat{u}}{\partial t} &= -i \omega \hat{\zeta},
\end{align*}
where $\omega = ck$ for a given wavenumber $k$. Also, from now on, write $u = \hat{u}$ and $\zeta = \hat{\zeta}$ to ease notation.

Chose a time-step $\Delta t$ and let $\alpha = \omega\Delta t = ck\Delta t$. The RK(3,2)-FB algorithm for this system is then given by:
\begin{align*}
    \bar{\zeta}^{n+1/3} &= \zeta^n - \frac{i \alpha}{3} u^n \\
    \bar{u}^{n+1/3} &= u^n - \frac{i \alpha}{3} \left( \beta_1\bar{\zeta}^{n+1/3} + (1 - \beta_1)\zeta^n \right) \\
    & \\
    \bar{\zeta}^{n+1/2} &= \zeta^n - \frac{i \alpha}{2} \bar{u}^{n+1/3} \\
    \bar{u}^{n+1/2} &= u^n - \frac{i \alpha}{2} \left( \beta_2 \bar{\zeta}^{n+1/2} + (1 - \beta_2)\zeta^n \right), \\
    & \\
    \zeta^{n+1} &= \zeta^n - i \alpha \bar{u}^{n+1/2} \\
    u^{n+1} &= u^n -i \alpha \left( \beta_3\zeta^{n+1} + (1-2\beta_3)\bar{\zeta}^{n+1/2} + \beta_3\zeta^n \right)
\end{align*}
where $\beta_i$ for $i = 1, 2, 3$, are the tunable FB feedback coefficients.

In [2]:
from sympy import *
from scipy.optimize import minimize, minimize_scalar
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# define relevent symbols 
#     _u = u^n, _zeta = \zeta^n
#     U1 = \bar{u}^{n+1/3}, ZETA1= \bar{\zeta}^{n+1/3}
#     U2 = \bar{u}^{n+1/2}, ZETA2= \bar{\zeta}^{n+1/2}
#     u = u^{n+1}, zeta = \zeta^{n+1}
_u, _zeta = symbols('u^n \zeta^n')
lam, alpha, beta1, beta2, beta3 = symbols('lambda alpha beta_1 beta_2 beta_3')

# define time-stepping scheme
ZETA1 = _zeta - (I*alpha/3) * _u
U1 = _u - (I*alpha/3) * ( beta1*ZETA1 + (1-beta1)*_zeta )

ZETA2 = _zeta - (I*alpha/2) * U1 
U2 = _u - (I*alpha/2) * ( beta2*ZETA2 + (1-beta2)*_zeta )

zeta = _zeta - I*alpha*U2
u = _u - I*alpha * ( beta3*zeta + (1-2*beta3)*ZETA2 + beta3*_zeta )

# simplify the expressions for zeta and u 
zeta  = zeta.simplify()
u = u.simplify()

Now, we have a system of linear equations in $\zeta^n$ and $u^n$. We can find the amplification matrix for the scheme by writing it in the form
$$ \mathbf{w}^{n+1} = G \mathbf{w}^n, $$
where $G$ is the amplification matrix and $\mathbf{w}^* = (\zeta^*, u^*)^T$.

We care about the amplification matrix $G$ as it follows from the above that $\mathbf{w}^n = G^n \mathbf{w}^0$. In turn, it follows that the solution $\mathbf{w}^n$ is bounded as $n \to \infty$ if and only if $||G^n||$ is bounded as $n \to \infty$.

In [3]:
# create the amplification matrix G
_w = [_zeta, _u]
w = [zeta, u]
G, _ = linear_eq_to_matrix(w, _w)

G

Matrix([
[                                        alpha**4*beta_2/12 - alpha**2/2 + 1,                                                                  -I*alpha**5*beta_1*beta_2/36 + I*alpha**3*beta_2/4 - I*alpha],
[-I*alpha**5*beta_2*beta_3/12 + I*alpha**3*beta_3/6 + I*alpha**3/6 - I*alpha, -alpha**6*beta_1*beta_2*beta_3/36 - alpha**4*beta_1*beta_3/9 + alpha**4*beta_1/18 + alpha**4*beta_2*beta_3/4 - alpha**2/2 + 1]])

Normally, we would now be interested in the eigenvalues of $G$ given that it can be shown that as long as $G$ is diagonalizable, then $||G^n||$ is bounded if and only if each eigenvalue $\lambda$ of $G$ is such that $|\lambda| \leq 1$.

However, we are first interesting in chosing the "optimal" FB feedback coefficients $\beta_i$.

To do this, we observe that we can write the orignal, Fourier transformed, system as
$$ \frac{\partial}{\partial t}\begin{pmatrix} \zeta \\ u \end{pmatrix} = \begin{pmatrix} 0 & -i\omega \\ -i\omega & 0 \end{pmatrix} \begin{pmatrix} \zeta \\ u \end{pmatrix}. $$

One can show that the solution to the system is
$$ \begin{pmatrix} \zeta(t) \\ u(t) \end{pmatrix} = e^{t \begin{pmatrix} 0 & -i\omega \\ -i\omega & 0 \end{pmatrix}} \begin{pmatrix} \zeta(0) \\ u(0) \end{pmatrix}, $$
and that eigenvalues of the matrix exponential above are
$$ \lambda = e^{\pm i \omega t}. $$

That is, the eigenvalues of the amplification matrix in the exact solution are $\lambda = e^{ \pm i \omega t}$. We would like the eigenvalues of our approximate amplification matrix $G$ to be the same, or as close as possible to $e^{\pm i \omega t}$.

Consider the solution to the system after exatly one time-step, at time $t = \Delta t$. We want the eigenvalues of $G$ to approximate $e^{\pm i \omega \Delta t} = e^{\pm i \alpha}$. 

To do this, we first calculate the characteristic polynomial $p(\lambda) = \text{det}(G - \lambda I)$ of $G$:

In [4]:
p = G.charpoly(lam).as_expr()
p

-alpha**6*beta_1*beta_2*beta_3/36 + alpha**6*beta_1*beta_2/36 + alpha**6*beta_1*beta_3/18 - alpha**6*beta_1/36 - alpha**4*beta_1*beta_3/9 + alpha**4*beta_1/18 + alpha**4*beta_2*beta_3/4 - alpha**4*beta_2/6 - alpha**4*beta_3/6 + alpha**4/12 + lambda**2 + lambda*(alpha**6*beta_1*beta_2*beta_3/36 + alpha**4*beta_1*beta_3/9 - alpha**4*beta_1/18 - alpha**4*beta_2*beta_3/4 - alpha**4*beta_2/12 + alpha**2 - 2) + 1

Next, we substitute in the desired value $\lambda = e^{\pm i \alpha}$ into the characteristic polynomial $p(\lambda)$ of $G$.

Now, if $e^{\pm i \alpha}$ was a true eigenvalue of $G$, then we would have
$$ p\left(e^{\pm i \alpha}\right) = 0. $$
This is not the case, so we want to choose values for $\beta$ and $\epsilon$ that get us as close as possible as $\alpha \to 0$ (i.e. as $\Delta t \to 0$).

So, expand the characteristic polynomial $p\left(e^{\pm i \alpha}\right)$ as a Taylor series in $\alpha$ centered at $0$:

In [5]:
exactPplus = p.subs([(lam, exp(I*alpha))])
exactPminus = p.subs([(lam, exp(-I*alpha))])

series(exactPplus, alpha, 0, 7)

alpha**4*(-beta_2/4 - beta_3/6 + 1/6) + alpha**5*(I*beta_1*beta_3/9 - I*beta_1/18 - I*beta_2*beta_3/4 - I*beta_2/12 + I/12) + alpha**6*(beta_1*beta_2/36 + beta_2*beta_3/8 + beta_2/24 - 2/45) + O(alpha**7)

In [6]:
series(exactPminus, alpha, 0, 7)

alpha**4*(-beta_2/4 - beta_3/6 + 1/6) + alpha**5*(-I*beta_1*beta_3/9 + I*beta_1/18 + I*beta_2*beta_3/4 + I*beta_2/12 - I/12) + alpha**6*(beta_1*beta_2/36 + beta_2*beta_3/8 + beta_2/24 - 2/45) + O(alpha**7)

So, we want that
$$\alpha^4 \left(-\frac{\beta_2}{4} - \frac{\beta_3}{6} + \frac{1}{6}\right) \pm i\alpha^5 \left( \frac{\beta_1\beta_3}{9} - \frac{\beta_1}{18} - \frac{\beta_2\beta_3}{4} - \frac{\beta_2}{12} + \frac{1}{12} \right) + \alpha^6\left( \frac{\beta_1\beta_2}{36} + \frac{\beta_2\beta_3}{8} + \frac{\beta_2}{24} - \frac{2}{45} \right) + \mathcal{O}(\alpha^7) \approx 0. $$

A local truncation error (LTE) analysis (see Section 3 of Darren's Overleaf document), shows us that RK(3,2)-FB is always at least 2nd order in time when applied to this problem, no matter the choice of FBFCs. However, the order can be increased with certain choices for $\beta_i$, in particular:
\begin{align*}
    \tau^n_u &= \begin{cases}
        \mathcal{O}(\Delta t^3) & \text{if } \beta_3 = 0 \\
        \mathcal{O}(\Delta t^4) & \text{if } \beta_3 = 0,\, \beta_1 = \frac{18}{24}
    \end{cases} \\
    & \\
    \tau^n_\zeta &= \mathcal{O}(\Delta t^3) \quad\text{if } \beta_2 = \frac{2}{3}
\end{align*}

For now, we are going to guess that taking $\beta_3 = 0$ is a bad call for CFL considerations since this removes all FB feedback from the final stage of the method. However, taking $\beta_2 = \frac{2}{3}$ *could* be a good call -- this reduces the dimension of our problem to 2 instead of 3. We could make some sort of argument along the line of "the $\zeta$ data are used in the FB steps to compute $u$, so it makes sense to increase their order"?

In [7]:
beta2Val = 2./3

series( nsimplify( exactPplus.subs([(beta2, beta2Val)]) ), alpha, 0, 7 )

-alpha**4*beta_3/6 + alpha**5*(I*beta_1*beta_3/9 - I*beta_1/18 - I*beta_3/6 + I/36) + alpha**6*(beta_1/54 + beta_3/12 - 1/60) + O(alpha**7)

In [8]:
series( nsimplify( exactPminus.subs([(beta2, beta2Val)]) ), alpha, 0, 7 )

-alpha**4*beta_3/6 + alpha**5*(-I*beta_1*beta_3/9 + I*beta_1/18 + I*beta_3/6 - I/36) + alpha**6*(beta_1/54 + beta_3/12 - 1/60) + O(alpha**7)

Working with the restriction that all $\beta_i$ are positive, the $\alpha^4$ term can't be touched, so we can try to eliminate the $\alpha^5$ term by expressing $\beta_3$ as a function of $\beta_1$:
$$\beta_3 = \frac{2\beta_1 - 1}{4\beta_1 - 6}$$
With this, we can try to eliminate the $\alpha^6$ term.

In [11]:
a6term = ( beta1/54 + beta3/12 - 1/60 ).subs([(beta3, (2*beta1 - 1)/(4*beta1 - 6))])
nsimplify(a6term)

beta_1/54 + (2*beta_1 - 1)/(12*(4*beta_1 - 6)) - 1/60

This cannot be solved by any real value of $\beta_1$, and the global minimum (absolute) value sits at $\beta_1 = 0$, which we decided is not allowed. There are no local minimums in the interval $(0,1)$, so our choice for $\beta_1$ becomes arbitrary -- this is a dead end...